In [34]:
import torch
import gc

from torch import nn
from torch.nn.utils import weight_norm
from torch.profiler import profile, record_function, ProfilerActivity

import timeit

torch.cuda.empty_cache()
gc.collect()

def WNConv1d(*args, **kwargs):
    return weight_norm(nn.Conv1d(*args, **kwargs))


conv1d = WNConv1d(128, 128, 7, dilation=1, padding=3).to('mps')

torch.manual_seed(0)
data = torch.randn(1, 128, 220500).to('mps')

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        conv1d(data)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
print("\n\n")
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

# # time
# execution_time = timeit.timeit(lambda: conv1d(data), number=1000)

# print(f"Average execution time: {execution_time / 1000:.6f} seconds per call")

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 model_inference        18.51%      81.697us       100.00%     441.363us     441.363us             1  
              aten::_weight_norm         0.49%       2.166us        55.58%     245.293us     245.293us             1  
    aten::_weight_norm_interface        53.29%     235.207us        55.09%     243.127us     243.127us             1  
                    aten::conv1d         0.66%       2.917us        25.91%     114.373us     114.373us             1  
               aten::convolution         1.43%       6.294us        25.25%     111.456us     111.456us             1  
              aten::_convolution         1.98%  

In [1]:
import tvm

from tvm import relax
from tvm.relax.frontend import nn

from typing import Optional
from tvm import te
from tvm import dlight as dl
from tvm.target import Target
import numpy as np
import tempfile

from mlc_dac.layers import WNConv1d

# conv1d = WNConv1d(128, 128, 7, dilation=1, padding=3)

conv1d = nn.Conv1D(128, 128, 7, dilation=1, padding=3)
mod, params = conv1d.export_tvm(
    {"forward": {"x": nn.spec.Tensor((1, 128, 220500), "float32")}}
)

trials = 2000
target = Target.from_device("metal")

with target, tempfile.TemporaryDirectory() as tmp_dir:
    seq = tvm.transform.Sequential(
        [
            # tvm.relax.transform.LegalizeOps(),
            # tvm.relax.transform.AnnotateTIROpPattern(),
            # tvm.relax.transform.FoldConstant(),
            # tvm.relax.transform.FuseOps(),
            # tvm.relax.transform.FuseTIR(),
            # dl.ApplyDefaultSchedule(
            #     # dl.gpu.Matmul(),
            #     dl.gpu.GEMV(),
            #     dl.gpu.Reduction(),
            #     dl.gpu.GeneralReduction(),
            #     dl.gpu.Fallback(),
            # ),

            relax.get_pipeline("zero"),
            relax.transform.MetaScheduleTuneTIR(work_dir=tmp_dir, max_trials_global=trials),
            relax.transform.MetaScheduleApplyDatabase(work_dir=tmp_dir),
        ]
    )

    mod = seq(mod)

mod.show()

2024-09-17 12:39:44 [INFO] Logging directory: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpsoeq5wyn/logs
2024-09-17 12:39:50 [INFO] LocalBuilder: max_workers = 24
2024-09-17 12:39:50 [INFO] LocalRunner: max_workers = 1
2024-09-17 12:39:51 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
2024-09-17 12:39:51 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-09-17 12:39:51 [INFO] [task_scheduler.cc:193] Sending 3 sample(s) to builder
2024-09-17 12:39:53 [INFO] [task_scheduler.cc:195] Sending 3 sample(s) to runner
2024-09-17 12:39:54 [DEBUG] XGB iter   0: tr-p-rmse: 0.422960	tr-a-peak@32: 1.000000	tr-rmse: 0.422972	tr-rmse: 0.422972
2024-09-17 12:39:54 [DEBUG] XGB iter  25: tr-p-rmse: 0.008021	tr-a-peak@32: 1.000000	tr-rmse: 0.008030	tr-rmse: 0.008030
2024-09-17 12:39:54 [DEBUG] XGB iter  50: tr-p-rmse: 0.003364	tr-a-peak@32: 1.000000	tr-rmse: 0.003361	tr-rmse: 0.003361
2024-09-17 12:39:54 [DEBUG] XGB iter  75: tr-p-rmse: 0.003362	tr-a-peak@32: 1.000

,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,1,1,0.0001,16.3717,16.3717,3,



Total trials: 3
Total latency (us): 16.3717

2024-09-17 12:39:55 [DEBUG] [task_scheduler.cc:318] 
 ID | Name | FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------
  0 | main |    1 |      1 |         0.0001 |      16.3717 |               16.3717 |      3 |      
---------------------------------------------------------------------------------------------------
Total trials: 3
Total latency (us): 16.3717

2024-09-17 12:39:55 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:193] Sending 0 sample(s) to builder
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:195] Sending 0 sample(s) to runner
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:237] [Updated] Task #0: "main"


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,1,1,0.0001,16.3717,16.3717,3,



Total trials: 3
Total latency (us): 16.3717

2024-09-17 12:39:55 [DEBUG] [task_scheduler.cc:318] 
 ID | Name | FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
---------------------------------------------------------------------------------------------------
  0 | main |    1 |      1 |         0.0001 |      16.3717 |               16.3717 |      3 |      
---------------------------------------------------------------------------------------------------
Total trials: 3
Total latency (us): 16.3717

2024-09-17 12:39:55 [INFO] [task_scheduler.cc:180] TaskScheduler picks Task #0: "main"
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:193] Sending 0 sample(s) to builder
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:195] Sending 0 sample(s) to runner
2024-09-17 12:39:55 [INFO] [task_scheduler.cc:237] [Updated] Task #0: "main"


...

,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,50605632000,1,10105.7572,5007.6042,5007.6042,2000,


2024-09-17 13:30:41 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50605632000 |      1 |     10105.7572 |    5007.6042 |             5007.6042 |   2000 |      
----------------------------------------------------------------------------------------------------------
Total trials: 2000
Total latency (us): 5007.6


Total trials: 2000
Total latency (us): 5007.6

2024-09-17 13:30:41 [INFO] [task_scheduler.cc:260] Task #0 has finished. Remaining task(s): 0


,Name,FLOP,Weight,Speed (GFLOPS),Latency (us),Weighted Latency (us),Trials,Done
0,main,50605632000,1,10105.7572,5007.6042,5007.6042,2000,Y


2024-09-17 13:30:41 [DEBUG] [task_scheduler.cc:318] 
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done 
----------------------------------------------------------------------------------------------------------
  0 | main | 50605632000 |      1 |     10105.7572 |    5007.6042 |             5007.6042 |   2000 |    Y 
----------------------------------------------------------------------------------------------------------
Total trials: 2000
Total latency (us): 5007.6


Total trials: 2000
Total latency (us): 5007.6



[13:30:41] /Users/cfruan/Documents/tvm-unity/src/relax/transform/meta_schedule.cc:119: Warning: Creating JSONDatabase. Workload at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpsoeq5wyn/database_workload.json, Tuning records at: /var/folders/50/mzqbqxqj5fddcby2mg3h334c0000gp/T/tmpsoeq5wyn/database_tuning_record.json


In [24]:
ex = relax.build(mod, target)
device = tvm.metal()

np.random.seed(0)
vm = relax.VirtualMachine(ex, device, profile=True)
tvm_data = tvm.nd.array(data.cpu(), device=device)
tvm_params = [np.random.randn(*param.shape).astype("float32") for _, param in params]
tvm_params = [tvm.nd.array(param, device=device) for param in tvm_params]

# output_tvm = vm["forward"](tvm_data, *tvm_params)
# output_tvm = output_tvm.asnumpy()

# output_tvm

report = vm.profile("forward", tvm_data, *tvm_params)
csv = report.csv()

with open("profile_conv1d.csv", "w", encoding="utf-8") as f:
    f.write(csv)
    print("Profile saved to profile_conv1d.csv")


Profile saved to profile_conv1d.csv


In [33]:
vm_eval = relax.VirtualMachine(ex, device)
timing_res = vm_eval.time_evaluator("forward", device, number=3, repeat=10, min_repeat_ms=100)(tvm_data, *tvm_params)
print(timing_res)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   6.4725       6.4723       6.5175       6.4398       0.0245                  


In [29]:
# with target:
#     seq = tvm.transform.Sequential(
#         [
#             dl.ApplyDefaultSchedule(
#                 dl.gpu.Matmul(),
#                 dl.gpu.GEMV(),
#                 dl.gpu.Reduction(),
#                 dl.gpu.GeneralReduction(),
#                 dl.gpu.Fallback(),
#             ),
#         ]
#     )

#     mod2 = seq(mod)

# ex2 = relax.build(mod2, target)
# device = tvm.metal()

# np.random.seed(0)
# tvm_data = tvm.nd.array(data.cpu(), device=device)
# tvm_params = [np.random.randn(*param.shape).astype("float32") for _, param in params]
# tvm_params = [tvm.nd.array(param, device=device) for param in tvm_params]

# # output_tvm = vm["forward"](tvm_data, *tvm_params)
# # output_tvm = output_tvm.asnumpy()

# # output_tvm

# vm_eval_2 = relax.VirtualMachine(ex2, device)
# timing_res = vm_eval_2.time_evaluator("forward", device, number=3, repeat=10, min_repeat_ms=100)(tvm_data, *tvm_params)
# print(timing_res)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   6.4132       6.4090       6.4972       6.3697       0.0340                  
